<a href="https://colab.research.google.com/github/guscldns/TestProject/blob/main/0623/06_3_KorNLI_%EB%8B%A4%EC%A4%91_%ED%81%B4%EB%9E%98%EC%8A%A4_%EB%B6%84%EB%A5%98(%EC%98%88%EC%8B%9C).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

References : https://github.com/kakaobrain/kor-nlu-datasets

# KorNLI 분류

In [ ]:
!pip install transformers
!pip install datasets

## 데이터셋 로드 및 구조 확인

In [ ]:
from datasets import load_dataset

cs = load_dataset("klue", "nli", split="train")
cs = cs.train_test_split(0.1)
test_cs = load_dataset("klue", "nli", split="validation")
train_cs = cs["train"]
valid_cs = cs["test"]

Generating train split:   0%|          | 0/24998 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/3000 [00:00<?, ? examples/s]

Dataset klue downloaded and prepared to /root/.cache/huggingface/datasets/klue/nli/1.0.0/e0fc3bc3de3eb03be2c92d72fd04a60ecc71903f821619cb28ca0e1e29e4233e. Subsequent calls will reuse this data.


In [ ]:
train_cs

Dataset({
    features: ['guid', 'source', 'premise', 'hypothesis', 'label'],
    num_rows: 22498
})

In [ ]:
valid_cs

Dataset({
    features: ['guid', 'source', 'premise', 'hypothesis', 'label'],
    num_rows: 2500
})

In [ ]:
test_cs

Dataset({
    features: ['guid', 'source', 'premise', 'hypothesis', 'label'],
    num_rows: 3000
})

## 데이터셋 전처리

In [ ]:
import pandas as pd
import numpy as np
import random
import time
import datetime
from tqdm import tqdm

import csv
import os

import tensorflow as tf

# BERT 사용을 위함
from transformers import BertTokenizer
from transformers import TFBertForSequenceClassification, AdamW, BertConfig
from transformers import get_linear_schedule_with_warmup

# for padding
from tensorflow.keras.preprocessing.sequence import pad_sequences

# 전처리 및 평가 지표
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score, roc_auc_score, accuracy_score, hamming_loss

훈련 데이터, 검증 데이터, 테스트 데이터에 대해서 `[CLS] 문장 [SEP]` 구조를 만듭니다. [CLS]는 분류를 하기 위해 BERT가 사용하는 첫번째 입력 토큰이며, [SEP]는 입력 문장의 종료를 나타내기 위해 사용하는 스페셜 토큰입니다.

In [ ]:
# ### 이 코드는 문장 구성을 보여드리기 위한 단순 예시 코드입니다!

# # 훈련 데이터, 검증 데이터, 테스트 데이터에 대해서 `[CLS] 문장 [SEP]` 구조를 만들어야 합니다.
# train_sentences = list(map(lambda train_cs: '[CLS] ' + str(train_cs['premise']) + ' [SEP] ' + str(train_cs['hypothesis']) + ' [SEP]', train_cs))
# validation_sentences = list(map(lambda valid_cs: '[CLS] ' + str(valid_cs['premise']) + ' [SEP] ' + str(valid_cs['hypothesis']) + ' [SEP]', valid_cs))
# test_sentences = list(map(lambda test_cs: '[CLS] ' + str(test_cs['premise']) + ' [SEP] ' + str(test_cs['hypothesis']) + ' [SEP]', test_cs))

In [ ]:
test_sentences[:5]

['[CLS] 흡연자분들은 발코니가 있는 방이면 발코니에서 흡연이 가능합니다. [SEP] 어떤 방에서도 흡연은 금지됩니다. [SEP]',
 '[CLS] 10명이 함께 사용하기 불편함없이 만족했다. [SEP] 10명이 함께 사용하기 불편함이 많았다. [SEP]',
 '[CLS] 10명이 함께 사용하기 불편함없이 만족했다. [SEP] 성인 10명이 함께 사용하기 불편함없이 없었다. [SEP]',
 '[CLS] 10명이 함께 사용하기 불편함없이 만족했다. [SEP] 10명이 함께 사용하기에 만족스러웠다. [SEP]',
 '[CLS] 10층에 건물사람들만 이용하는 수영장과 썬베드들이 있구요. [SEP] 건물사람들은 수영장과 썬베드를 이용할 수 있습니다. [SEP]']

In [ ]:
train_labels = train_cs['label']
validation_labels = valid_cs['label']
test_labels = test_cs['label']

얽힘(entailment) : 0
중립(neutral) : 1
모순(contradiction) : 2

## BERT 토크나이저를 이용한 전처리

BERT를 사용하기 위해서는 토크나이저와 모델이 반드시 맵핑 관계여야만 합니다. 다시 말해 아래의 이름에 들어가는 모델이름은 반드시 동일해야 합니다.

* BertTokenizer.from_pretrained('모델이름')
* BertForSequenceClassification.from_pretrained("모델이름")

토크나이저는 내부적으로 Vocabulary를 갖고 있어 정수 인코딩을 수행해주는 모듈입니다.

In [ ]:
# 한국어 BERT 중 하나인 'klue/bert-base'를 사용.
tokenizer = BertTokenizer.from_pretrained('klue/bert-base')

In [ ]:
X_train = tokenizer(train_cs['premise'], train_cs['hypothesis'], padding=True, truncation=True, return_tensors = 'tf')
X_valid = tokenizer(valid_cs['premise'], valid_cs['hypothesis'], padding=True, truncation=True, return_tensors = 'tf')
X_test = tokenizer(test_cs['premise'], test_cs['hypothesis'], padding=True, truncation=True, return_tensors = 'tf')

In [ ]:
X_train

In [ ]:
import tensorflow as tf

# 주어진 데이터소스를 여러 Tensor로 자른 후 iterator(반복가능 객체)로 만들기
train_dataset = tf.data.Dataset.from_tensor_slices((
    dict(X_train),
    train_labels
))

val_dataset = tf.data.Dataset.from_tensor_slices((
    dict(X_valid),
    validation_labels
))

test_dataset = tf.data.Dataset.from_tensor_slices((
    dict(X_test),
    test_labels
))

훈련 데이터, 검증 데이터, 텍스트 데이터에 대해서 data_to_tensor 함수를 통해서   정수 인코딩 된 데이터, 레이블, 어텐션 마스크를 얻습니다.

In [ ]:
list(train_dataset.take(1))

[({'input_ids': <tf.Tensor: shape=(100,), dtype=int32, numpy=
   array([    2,   648,  1370,  2522,  8362,  4622,  3810,    25,  2211,
           2119,  3655,  5927,  4192,  6233,  4153,  2170, 11391,  2062,
             18,     3,  8362,  4622,  3810,  2119,  4153,  2170,  7651,
           2951,  2062,    18,     3,     0,     0,     0,     0,     0,
              0,     0,     0,     0,     0,     0,     0,     0,     0,
              0,     0,     0,     0,     0,     0,     0,     0,     0,
              0,     0,     0,     0,     0,     0,     0,     0,     0,
              0,     0,     0,     0,     0,     0,     0,     0,     0,
              0,     0,     0,     0,     0,     0,     0,     0,     0,
              0,     0,     0,     0,     0,     0,     0,     0,     0,
              0,     0,     0,     0,     0,     0,     0,     0,     0,
              0], dtype=int32)>,
   'token_type_ids': <tf.Tensor: shape=(100,), dtype=int32, numpy=
   array([0, 0, 0, 0, 0, 0, 0, 0, 0

In [ ]:
print('훈련 데이터의 크기:', len(train_labels))
print('검증 데이터의 크기:', len(validation_labels))
print('테스트 데이터의 크기:', len(test_labels))

훈련 데이터의 크기: 22498
검증 데이터의 크기: 2500
테스트 데이터의 크기: 3000


## GPU가 정상 셋팅되었는지 확인.
Colab에서 GPU를 사용하기 위해서는 아래와 같이 설정이 되어있어야만 합니다.

* 런타임 > 런타임 유형 변경 > 하드웨어 가속기 > 'GPU' 선택

## 모델 로드하기

BERT를 사용하여 텍스트를 분류하는 BERT 아키텍처는 `BertForSequenceClassification.from_pretrained("모델이름")`을 넣어서 가능합니다. 레이블 수로 num_labels라는 인자값에 레이블의 수를 기재해줍니다.

In [ ]:
from transformers import TFBertModel

class MyModel(tf.keras.Model):
    def __init__(self, model_name, num_labels):
        super().__init__()
        self.bert = TFBertModel.from_pretrained(model_name, from_pt=True)
        self.classifier = tf.keras.layers.Dense(num_labels,
                                                kernel_initializer=tf.keras.initializers.TruncatedNormal(0.02),
                                                activation='softmax',
                                                name='classifier')

    def call(self, input_ids = None, attention_mask=None, token_type_ids=None):
        # input_ids, attention_mask, token_type_ids = inputs
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask, token_type_ids=token_type_ids)
        cls_token = outputs[1]
        prediction = self.classifier(cls_token)

        return prediction


## 모델 학습

In [ ]:
tf.keras.backend.clear_session()

mymodel = MyModel("klue/bert-base", num_labels=3)
optimizer = tf.keras.optimizers.Adam(learning_rate=5e-5)
loss = tf.keras.losses.SparseCategoricalCrossentropy()
mymodel.compile(optimizer=optimizer, loss=loss, metrics = ['accuracy'])

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'bert.embeddings.position_ids', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.bias', 'cls.predictions.decoder.weight', 'cls.predictions.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the PyTorch model.
If your task is similar to the 

In [ ]:
# hugging face - built in ver
num_labels = 3
model = TFBertForSequenceClassification.from_pretrained("klue/bert-base", num_labels=num_labels,
                                                        from_pt = True)
optimizer = tf.keras.optimizers.Adam(learning_rate=5e-5)

loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits = True)
model.compile(optimizer=optimizer, loss=loss, metrics = ['accuracy'])

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertForSequenceClassification: ['bert.embeddings.position_ids']
- This IS expected if you are initializing TFBertForSequenceClassification from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertForSequenceClassification from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
Some weights or buffers of the TF 2.0 model TFBertForSequenceClassification were not initialized from the PyTorch model and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
model.summary()

Model: "tf_bert_for_sequence_classification_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 bert (TFBertMainLayer)      multiple                  110617344 
                                                                 
 dropout_112 (Dropout)       multiple                  0         
                                                                 
 classifier (Dense)          multiple                  2307      
                                                                 
Total params: 110,619,651
Trainable params: 110,619,651
Non-trainable params: 0
_________________________________________________________________


In [ ]:
# tf.keras.utils.plot_model(model, show_shapes = True)

In [ ]:
model.layers

In [ ]:
model.layers[0].config # huggingface 객체
model.layers[0].get_config() # tensorflow 객체

In [ ]:
model.layers[2].get_config()

{'name': 'classifier',
 'trainable': True,
 'dtype': 'float32',
 'units': 3,
 'activation': 'linear',
 'use_bias': True,
 'kernel_initializer': {'class_name': 'TruncatedNormal',
  'config': {'mean': 0.0, 'stddev': 0.02, 'seed': None}},
 'bias_initializer': {'class_name': 'Zeros', 'config': {}},
 'kernel_regularizer': None,
 'bias_regularizer': None,
 'activity_regularizer': None,
 'kernel_constraint': None,
 'bias_constraint': None}

In [ ]:
# y_true = [1, 2]
# y_pred_logits = tf.constant([[0.1, 10, 1], [1, 100, 2]], dtype = tf.float32)

# # tf.keras.layers.Softmax()(y_pred_logits)

# # y_pred = [[0.05, 0.95, 0], [0.1, 0.8, 0.1]]


# scce = tf.keras.losses.SparseCategoricalCrossentropy(from_logits = True)
# scce(y_true, y_pred_logits)


In [ ]:
from tensorflow.keras.callbacks import EarlyStopping

early_stopping = EarlyStopping(
    monitor="val_accuracy",
    min_delta=0.001,
    patience=2)

model.fit(
    train_dataset.shuffle(10000).batch(32),
    epochs=50,
    validation_data = val_dataset.shuffle(len(validation_labels)).batch(32),
    callbacks = [early_stopping]
)

Epoch 1/50
704/704 [==============================] - 142s 130ms/step - loss: 0.5872 - accuracy: 0.7577 - val_loss: 0.4628 - val_accuracy: 0.8120
Epoch 2/50
704/704 [==============================] - 61s 87ms/step - loss: 0.2982 - accuracy: 0.8904 - val_loss: 0.5406 - val_accuracy: 0.8196
Epoch 3/50
704/704 [==============================] - 61s 87ms/step - loss: 0.1617 - accuracy: 0.9438 - val_loss: 0.5724 - val_accuracy: 0.8268
Epoch 4/50
704/704 [==============================] - 61s 87ms/step - loss: 0.1044 - accuracy: 0.9633 - val_loss: 0.6247 - val_accuracy: 0.8340
Epoch 5/50
704/704 [==============================] - 60s 85ms/step - loss: 0.0711 - accuracy: 0.9769 - val_loss: 0.6718 - val_accuracy: 0.8296
Epoch 6/50
704/704 [==============================] - 59s 84ms/step - loss: 0.0657 - accuracy: 0.9787 - val_loss: 0.8323 - val_accuracy: 0.8280


# 예측

In [ ]:
text = {"text" : "흡연자분들은 발코니가 있는 방이면 발코니에서 흡연이 가능합니다.", "text_pair" : "어떤 방에서도 흡연은 금지됩니다."}
inputs = tokenizer(text['text'], text['text_pair'], return_tensors = 'tf')

inputs

{'input_ids': <tf.Tensor: shape=(1, 30), dtype=int32, numpy=
array([[    2, 25313,  2377,  2031,  2073, 20812,  2116,  1513,  2259,
         1129, 24094, 20812, 27135,  9753,  2052,  3662, 11800,    18,
            3,  3711,  1129, 27135,  2119,  9753,  2073,  5040,  3598,
         3606,    18,     3]], dtype=int32)>, 'token_type_ids': <tf.Tensor: shape=(1, 30), dtype=int32, numpy=
array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1]], dtype=int32)>, 'attention_mask': <tf.Tensor: shape=(1, 30), dtype=int32, numpy=
array([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1]], dtype=int32)>}

In [ ]:
np.argmax(model(**inputs))

2